In [4]:
import pandas as pd
from google.cloud import bigquery
from pandas_gbq import gbq
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'key.json'

In [5]:
bigquery_client = bigquery.Client()
query = "SELECT * FROM `lab1-449822.walmartsales.sales` limit 10000"
query_result = bigquery_client.query(query)
df_sales = query_result.to_dataframe()

C:\Users\apoor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [6]:
df_sales.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,SalesDate,TransactionNumber
0,3517714,1,590,1,1,0.0,2018-01-09 00:49:28.020000+00:00,LAH3Z9FMK2U7XGGERFJ5
1,4944766,1,1683,1,1,0.0,2018-01-04 07:10:25.020000+00:00,019O1RWHNJWUKS31VQ9G
2,1542841,9,1940,1,1,0.0,2018-04-30 11:41:34.690000+00:00,HJHZNCIYT7NLZUQ3IWJU
3,159791,9,3083,1,1,0.0,2018-01-14 02:22:20.060000+00:00,YHYEV1IK5883R9L76AA3
4,5358168,2,3121,1,1,0.0,2018-01-12 17:32:19.540000+00:00,18S9KBE5PCP8JDBESU9D


In [7]:
df_sales['SalesDate'] = pd.to_datetime(df_sales['SalesDate']).dt.date
df_sales.head()


,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,SalesDate,TransactionNumber
0,3517714,1,590,1,1,0.0,2018-01-09,LAH3Z9FMK2U7XGGERFJ5
1,4944766,1,1683,1,1,0.0,2018-01-04,019O1RWHNJWUKS31VQ9G
2,1542841,9,1940,1,1,0.0,2018-04-30,HJHZNCIYT7NLZUQ3IWJU
3,159791,9,3083,1,1,0.0,2018-01-14,YHYEV1IK5883R9L76AA3
4,5358168,2,3121,1,1,0.0,2018-01-12,18S9KBE5PCP8JDBESU9D


In [8]:
try:
    df_sales.to_gbq(
    destination_table='walmartsales.sales_final', 
    project_id='lab1-449822',            
    if_exists='fail',)
    
except gbq.TableCreationError as e:
    if "Already Exist" in str(e):
        print("The Table Already Exists")
    else:
        raise
    


C:\Users\apoor\AppData\Local\Temp\ipykernel_19080\1341140065.py:2: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_sales.to_gbq(


The Table Already Exists


In [9]:
with open('final_query.sql', 'r') as file:
   joined_query = file.read()
joined_result = bigquery_client.query(joined_query)
result = joined_result.to_dataframe()
result.head()

C:\Users\apoor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,SalesID,CustomerID,ProductID,quantity,Discount,SalesDate,CityName,CountryName,ProductName,Price,Class,Resistant,CategoryName
0,4891123,130,1,1,20.0,2018-04-06 21:55:24.330000+00:00,Lincoln,United States,Flour - Whole Wheat,74.2988,Medium,Durable,Cereals
1,2867343,181,1,1,10.0,2018-01-05 04:03:07.980000+00:00,San Francisco,United States,Flour - Whole Wheat,74.2988,Medium,Durable,Cereals
2,774720,517,1,1,0.0,2018-04-17 13:29:31.970000+00:00,Corpus Christi,United States,Flour - Whole Wheat,74.2988,Medium,Durable,Cereals
3,2952728,549,1,1,5.0,2018-01-05 07:39:35.190000+00:00,Miami,United States,Flour - Whole Wheat,74.2988,Medium,Durable,Cereals
4,1151881,624,1,1,5.0,2018-05-06 17:00:01.670000+00:00,Atlanta,United States,Flour - Whole Wheat,74.2988,Medium,Durable,Cereals


In [10]:
result['SalesDate'] = pd.to_datetime(result['SalesDate']).dt.date
result.head()

,SalesID,CustomerID,ProductID,quantity,Discount,SalesDate,CityName,CountryName,ProductName,Price,Class,Resistant,CategoryName
0,4891123,130,1,1,20.0,2018-04-06,Lincoln,United States,Flour - Whole Wheat,74.2988,Medium,Durable,Cereals
1,2867343,181,1,1,10.0,2018-01-05,San Francisco,United States,Flour - Whole Wheat,74.2988,Medium,Durable,Cereals
2,774720,517,1,1,0.0,2018-04-17,Corpus Christi,United States,Flour - Whole Wheat,74.2988,Medium,Durable,Cereals
3,2952728,549,1,1,5.0,2018-01-05,Miami,United States,Flour - Whole Wheat,74.2988,Medium,Durable,Cereals
4,1151881,624,1,1,5.0,2018-05-06,Atlanta,United States,Flour - Whole Wheat,74.2988,Medium,Durable,Cereals


In [11]:
try:
    result.to_gbq(
    destination_table='walmartsales.joined_final', 
    project_id='lab1-449822',            
    if_exists='fail' 
    )
except gbq.TableCreationError as e:
    if "Already Exists" in str(e):
        print("Table Already Exists")
    else:
        raise e

C:\Users\apoor\AppData\Local\Temp\ipykernel_19080\2656026326.py:2: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  result.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 19508.39it/s]
